# Import package

In [1]:
import numpy as np
import pyjet
from pyjet import cluster
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import uproot


# Define function

In [2]:
def flatten(event): # for single event in order to reshape data
    fp = np.concatenate((np.expand_dims(event[0], axis=-1),
                         np.expand_dims(event[1], axis=-1),
                         np.expand_dims(event[2], axis=-1),
                         np.expand_dims(event[3], axis=-1)), axis=-1)
    fp = fp.transpose((1,0))
    fp = np.core.records.fromarrays( [fp[:][0],fp[:][1],fp[:][2],fp[:][3]], names= 'pT, eta, phi, mass' , formats = 'f8, f8, f8,f8')

    return fp

def jet_clustering(event, R0, p = -1):
    # R0 = Clustering radius for the main jets
    flattened_event = flatten(event)
    ## p = -1, 0, 1 => anti-kt, C/A, kt Algorithm
    sequence = cluster(flattened_event, R=R0, p= p)
    # List of jets
    jets = sequence.inclusive_jets()
    return jets



# Load data

In [3]:
# type your root file here
root_file = '/Storage/james/ROOT/SVJ/rinv0/test0.root'
#use uproot 
file = uproot.open(root_file)
# use calorimeter information to reconstruct jet
events = np.array([np.array(file["Delphes;1"]["Tower.ET"].array()),
                   np.array(file["Delphes;1"]["Tower.Eta"].array()),
                   np.array(file["Delphes;1"]["Tower.Phi"].array()),
                   np.array(file["Delphes;1"]["Tower.E"].array())*0  #assume m<<1
                  ])

# just reshape data
events = np.expand_dims(events, axis=-1)
events = events.transpose((1,0,2))
events = np.squeeze(events,axis=(2,))


# Loop over all event to get jets in each event

In [4]:
jets = []
for event in tqdm(events):
    jet = jet_clustering(event, 0.4, p = -1) # anti-kt jet with R = 0.4
    jets.append(jet)

100%|██████████| 30000/30000 [00:12<00:00, 2393.18it/s]


# Show jet

In [5]:
nth_event = 0
ith_jet_in_event = 0

jet = jets[nth_event][ith_jet_in_event]
jet

PseudoJet(pt=595.870, eta=-0.192, phi=1.642, mass=74.282)

## See the function in jet 

In [6]:
dir(jet)

['__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'area',
 'child',
 'constituents',
 'constituents_array',
 'e',
 'et',
 'eta',
 'mass',
 'parents',
 'phi',
 'pt',
 'px',
 'py',
 'pz',
 'userinfo']

In [14]:
print("{:^8}{:^20}{:^20}{:^20}".format("Index", "pT", "eta", "phi"))
print("{:^8}{:^20}{:^20}{:^20}".format(0, round(jet.pt, 4), round(jet.eta,4), round(jet.phi, 4) ))


 Index           pT                 eta                 phi         
   0          595.8702            -0.1924              1.6425       
